In [1]:
from tensor import Tensor, TensorSpec, TensorShape
from utils.index import Index
from algorithm import vectorize
from sys.info import simdwidthof
from random import rand
import math

let input_ids = rand[DType.uint32](4)
print(input_ids)

Tensor([[3163445217, 524636540, 4176527650, 3125866584]], dtype=uint32, shape=4)


In [48]:
# TODO: implement
fn get_embedding(input_ids: Tensor[DType.uint32]) -> Tensor[DType.float32]:
    # input_ids:  Tensor of shape (b, 1)
    # return:     Tensor of shape (b, 1, d_model)
    let EMBEDDING_DIM = 4
    return rand[DType.float32](EMBEDDING_DIM)

print(get_embedding(input_ids))

Tensor([[0.19650378823280334, 0.14129260182380676, 0.66934406757354736, 0.80183345079421997]], dtype=float32, shape=4)


In [26]:
# TODO: implement this
fn compute_layer(input: Tensor[DType.float32]) -> Tensor[DType.float32]:
    # return a random f32 tensor for now
    return rand[DType.float32](input.shape())

In [68]:
fn RMS_norm(input: Tensor[DType.float32]) -> Tensor[DType.float32]:
    var mean: Float32 = 0.0
    var squares = Tensor[DType.float32](input.shape())
    var output = Tensor[DType.float32](input.shape())
    alias simd_float32_width: Int = simdwidthof[DType.float32]()
    
    # Vectorized squaring of input
    @parameter
    fn square_tensor[simd_float32_width: Int](idx: Int) -> None:
        squares.simd_store[simd_float32_width]
                (idx, math.pow(input.simd_load[simd_float32_width](idx), 2))
    vectorize[simd_float32_width, square_tensor](input.num_elements())

    # Sum up squares
    @parameter
    fn sum_squares[simd_float32_width: Int](idx: Int) -> None:
        mean += squares.simd_load[simd_float32_width](idx).reduce_add()
    vectorize[simd_float32_width, sum_squares](squares.num_elements())

    mean /= squares.num_elements()
    
    let rms: Float32 = math.sqrt(mean)
    @parameter
    fn divide_by_rms[simd_float32_width: Int](idx: Int) -> None:
        output.simd_store[simd_float32_width](idx, input.simd_load[simd_float32_width](idx) / rms)
    vectorize[simd_float32_width, divide_by_rms](output.num_elements())

    return output

In [62]:
# TODO: implement this
fn lm_head(input: Tensor[DType.float32]) -> Tensor[DType.float32]:
    # return a random f32 tensor for now
    return rand[DType.float32](input.shape())

In [50]:
# TODO: implement this
fn forward_pass(input_ids: Tensor[DType.uint32]) -> Tensor[DType.float32]:
    let num_layers = 2
    var x = get_embedding(input_ids)
    
    for i in range(num_layers):
        x = compute_layer(x)
            
    let normalized = RMS_norm(x)
    let logits = lm_head(normalized)

    return logits